<a href="https://colab.research.google.com/github/dhanu902/FoodieChat-Bot/blob/main/BOT_MODEL_BERT_BasedIntentClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 Intent Classification with BERT

---

## 💡 What is **Intent Classification**?

**Intent Classification** is a task in Natural Language Processing (NLP) where the model predicts the **purpose or goal** of a user's input.

---

### 🧑‍💻 Example

| User Input                            | Predicted Intent       |
|--------------------------------------|------------------------|
| "I want to book a flight to London." | `book_flight`          |
| "What's the weather in Colombo?"     | `get_weather`          |
| "Cancel my reservation."             | `cancel_reservation`   |
| "Play some relaxing music."          | `play_music`           |

---

## ⚙️ Why Use BERT for Intent Classification?

BERT is effective because it:
- Understands **context** using bidirectional attention.
- Deals well with **variations in input**, such as slang and paraphrasing.
- Handles **long and complex sentences** better than traditional models.

---

## 🔁 Typical Workflow for BERT-based Intent Classifier

1. **Input**: Raw user query  
2. **Preprocessing**: Tokenize using `BertTokenizer`
3. **Model**: Fine-tune `BERT` with a classification head (`BertForSequenceClassification`)
4. **Output**: Intent label (e.g., `book_flight`, `cancel_reservation`)

---

## 📊 Evaluation Metrics for Intent Classification

Use the following **classification metrics** to evaluate your model:

| Metric           | Description                                                                 |
|------------------|-----------------------------------------------------------------------------|
| `Accuracy`       | Proportion of correctly predicted intents.                                  |
| `Precision`      | TP / (TP + FP) — How many predicted intents were actually correct?          |
| `Recall`         | TP / (TP + FN) — How many actual intents were correctly predicted?          |
| `F1 Score`       | Harmonic mean of precision and recall.                                      |
| `Classification Report` | Includes precision, recall, F1-score per intent, and support count. |

---

### ✅ Example Code for Evaluation

```c
from sklearn.metrics import classification_report, accuracy_score

# Predict intents
y_val_pred = model.predict(X_val)

# Accuracy
print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))

# Label names
target_names = label_encoder.classes_

# Classification Report
print(classification_report(y_val, y_val_pred, target_names=target_names))


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# <----- Load Data ----->
import pandas as pd

df_train = pd.read_csv('/content/drive/MyDrive/ChatBot/Preprocessed/intent_train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/ChatBot/Preprocessed/intent_test.csv')
df_val = pd.read_csv('/content/drive/MyDrive/ChatBot/Preprocessed/intent_val.csv')

In [3]:
# <----- safe drop of NULL values ----->
df_train.dropna(subset=['text'], inplace=True)
df_test.dropna(subset=['text'], inplace=True)
df_val.dropna(subset=['text'], inplace=True)

In [4]:
# <----- Encoding ----->
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_train['label'] = le.fit_transform(df_train['label'])
df_test['label'] = le.transform(df_test['label'])
df_val['label'] = le.transform(df_val['label'])

num_labels = len(le.classes_)

In [5]:
# <----- Tokenize - BERT tokenizer ----->
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def Tokenize(batch):
  return tokenizer(batch['text'],
                   padding='max_length',
                   truncation=True,
                   max_length=64)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [6]:
# <----- Convert to Hugging face Dataset ----->
from datasets import Dataset

train_dataset = Dataset.from_pandas(df_train[['text', 'label']])
test_dataset = Dataset.from_pandas(df_test[['text', 'label']])
val_dataset = Dataset.from_pandas(df_val[['text', 'label']])

train_dataset = train_dataset.map(Tokenize, batched=True)
test_dataset = test_dataset.map(Tokenize, batched=True)
val_dataset = val_dataset.map(Tokenize, batched=True)

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/38540 [00:00<?, ? examples/s]

Map:   0%|          | 0/8258 [00:00<?, ? examples/s]

Map:   0%|          | 0/8265 [00:00<?, ? examples/s]

In [7]:
# <----- Load Evaluation Metrics ----->
from sklearn.metrics import accuracy_score, f1_score
from transformers import EvalPrediction
import numpy as np

def Compute_Metrics(eval_pred: EvalPrediction):

  logits, labels = eval_pred
  preds = np.argmax(logits, axis=1)

  return {
      "Accuracy": accuracy_score(labels, preds),
      "F1": f1_score(labels, preds, average='weighted')
  }

In [8]:
# <----- Param Grid ----->
from itertools import product

param_grid = {
    'epochs': [2, 3],
    'train_batch': [16, 32],
    'eval_batch': [32],
    'lr': [2e-5, 3e-5]
}

grid = list(product(param_grid['epochs'],
                    param_grid['train_batch'],
                    param_grid['eval_batch'],
                    param_grid['lr']))

In [ ]:
# <----- Define Training Arguments ----->
from transformers import BertForSequenceClassification, TrainingArguments, Trainer
from itertools import product
import pandas as pd

results = []

for epoch, train_batchsize, eval_batchsize, lr in grid:
  print(f"\nTraining config: Epochs = {epoch}, batch: {train_batchsize}, lr: {lr}")

  model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

  training_args = TrainingArguments(
    output_dir = './temp',              ## --> temp = No model will be saved. Temporary saved
    num_train_epochs = epoch,
    per_device_train_batch_size = train_batchsize,
    per_device_eval_batch_size = eval_batchsize,
    eval_strategy = 'epoch',
    save_strategy = 'no',               ## --> no - Disable saving
    learning_rate = lr,
    logging_steps = 10,
    report_to = 'none',
    disable_tqdm = True
    )

  trainer = Trainer (
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    compute_metrics = Compute_Metrics,
    tokenizer = tokenizer
    )

  # <----- Model Train ----->
  trainer.train()
  metrics = trainer.evaluate(val_dataset)

  results.append({
      'epoch': epoch,
      'train_batchsize': train_batchsize,
      'eval_batchsize': eval_batchsize,
      'learning_rate': lr,
      'val_accuracy': round(metrics['eval_Accuracy'], 4),
      'val_f1': round(metrics['eval_F1'], 4)
  })


Training config: Epochs = 2, batch: 16, lr: 2e-05


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2939695315.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer (
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 2.3268, 'grad_norm': 6.743325233459473, 'learning_rate': 1.9962640099626403e-05, 'epoch': 0.004151100041511001}
{'loss': 2.2033, 'grad_norm': 5.174942970275879, 'learning_rate': 1.9921129099211293e-05, 'epoch': 0.008302200083022002}
{'loss': 2.1174, 'grad_norm': 5.4011735916137695, 'learning_rate': 1.9879618098796183e-05, 'epoch': 0.012453300124533}
{'loss': 2.027, 'grad_norm': 4.702395439147949, 'learning_rate': 1.9838107098381072e-05, 'epoch': 0.016604400166044003}
{'loss': 1.876, 'grad_norm': 5.726657390594482, 'learning_rate': 1.9796596097965962e-05, 'epoch': 0.020755500207555}
{'loss': 1.8798, 'grad_norm': 6.220581531524658, 'learning_rate': 1.9755085097550852e-05, 'epoch': 0.024906600249066}
{'loss': 1.7814, 'grad_norm': 6.016400337219238, 'learning_rate': 1.9713574097135742e-05, 'epoch': 0.029057700290577002}
{'loss': 1.6314, 'grad_norm': 7.42952299118042, 'learning_rate': 1.9672063096720632e-05, 'epoch': 0.033208800332088007}
{'loss': 1.4768, 'grad_norm': 7.39373350143

In [ ]:
# <----- Save ----->

results_df = pd.DataFrame(results)
results_df.to_csv('/content/drive/MyDrive/ChatBot/Preprocessed/results.csv', index=False)

print("\n Parameter sweep complete. Summery: ")
print(results_df)